# Day 81: Clinical Decision Support Safety

Clinical Decision Support (CDS) AI assists healthcare providers in making diagnostic and treatment decisions. In this high-stakes domain, 'Safety' means ensuring the AI recognizes its own limitations, respects medical constraints (contraindications), and defaults to human expertise in emergencies.

In this lab, we implement:
1. **Uncertainty Calibration**: Flagging diagnoses where the model has low confidence.
2. **Contraindication Checking**: Cross-referencing proposed treatments against patient history (e.g., drug-drug interactions).
3. **Protocol Enforcement**: Ensuring urgent symptoms (like chest pain) trigger mandatory emergency routes.

In [ ]:
import sys
import os

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.assurance.clinical_safety import ClinicalSupportSafety, ClinicalSuggestion

## 1. Safety Check: Uncertainty

The model suggests a diagnosis but with only 50% confidence. The safety layer must flag this as 'unreliable'.

In [ ]:
safety = ClinicalSupportSafety(uncertainty_threshold=0.8)

suggestion = ClinicalSuggestion(
    diagnosis="Common Cold", 
    confidence=0.5, 
    recommended_action="Rest", 
    contraindications=[]
)

evaluation = safety.evaluate_suggestion(suggestion, [])
print(f"Is Safe: {evaluation['is_safe']}")
print(f"Warnings: {evaluation['warnings']}")

## 2. Safety Check: Contraindications

The model suggests Warfarin (a blood thinner), but the patient is already taking Aspirin. This is a dangerous combination.

In [ ]:
history = ["Currently taking Aspirin for headaches", "No allergies"]
suggestion = ClinicalSuggestion(
    diagnosis="Blood Clot Prevention", 
    confidence=0.95, 
    recommended_action="Prescribe Warfarin", 
    contraindications=[]
)

evaluation = safety.evaluate_suggestion(suggestion, history)
print(f"Is Safe: {evaluation['is_safe']}")
for w in evaluation['warnings']:
    print(f" [!] {w}")

## 3. Policy Enforcement: Emergency Handling

Safety protocols often require immediate human intervention for specific symptoms, regardless of AI confidence.

In [ ]:
emergency = ClinicalSuggestion(
    diagnosis="Potential Myocardial Infarction (Chest Pain)", 
    confidence=0.9, 
    recommended_action="Schedule follow-up next week", 
    contraindications=[]
)

evaluation = safety.evaluate_suggestion(emergency, [])
print(f"Recommended Mitigation: {evaluation['action']}")
print(f"Safety Warning: {evaluation['warnings'][0]}")